In [56]:
#!pip install nltk


# Importing Libraries

In [199]:
import ast
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches # For fuzzy Match

# Loading Dataset

In [200]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [201]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [202]:
movies.shape

(4803, 20)

In [203]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [204]:
credits.shape

(4803, 4)

In [205]:
credits.head(1)['cast']

0    [{"cast_id": 242, "character": "Jake Sully", "...
Name: cast, dtype: object

In [206]:
# credits.head(1)['cast'].values

# Merging Dataframes

In [207]:
# merging both Dataframes  (credits, movies)

movies = movies.merge(credits, on='title')

In [208]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [209]:
movies.shape

(4809, 23)

In [210]:
#  taking important columns

movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [211]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


## Preprocessing

In [212]:
movies.shape

(4809, 7)

In [213]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [214]:
# Removing duplicates

movies.dropna(inplace= True)

In [215]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [216]:
# Checking for duplicated values

movies.duplicated().sum()

0

# Preprocessing columns content

In [217]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [218]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [219]:
# the convert function is used to convert a JSON-like string (from the CSV) into a Python list of names.

import ast

def convert(obj):
    list = []
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list



In [220]:
movies['genres'] = movies['genres'].apply(convert)

In [221]:
movies['keywords'] = movies['keywords'].apply(convert)

In [222]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [223]:
# Used for the cast column.
# Movies can have dozens of cast members, but usually you don’t need all of them.
# convert3 extracts only the top 3 actor names.

def convert3(obj):
    result = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            result.append(i['name'])
            counter += 1
        else:
            break
    return result


In [224]:
movies['cast'] = movies['cast'].apply(convert3)

In [225]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [226]:
# movies['crew'][0]

In [227]:
# for movie recommendation, usually only the director is important, so fetching only the director from "crew" column.

def fetch_director(obj):
    list = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list.append(i['name'])
            break
    return list
            

In [228]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [229]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [230]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [231]:
#  converting overview string in list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [232]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


# removes spaces inside multi-word tokens.

Remove spaces so **multi-word terms (like “Science Fiction” or “James Cameron”) become single tokens (“ScienceFiction”, “JamesCameron”)**.

👉 This prevents **CountVectorizer** from splitting them into separate, meaningless words and keeps each genre/person as one unique feature for better recommendations.


In [233]:
# removes spaces inside multi-word tokens.

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [234]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [235]:
# Creating Tags col

movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [236]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."


# Creating new DataFrame


In [237]:
new_df = movies[['movie_id', 'title', 'tags']]

In [238]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\PC\AppData\Local\Temp\ipykernel_6776\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [239]:
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."


In [240]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [241]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df['tags'][0]

C:\Users\PC\AppData\Local\Temp\ipykernel_6776\3982020389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron'

In [242]:
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [243]:
new_df['tags'][1]

'captain barbossa, long believed to be dead, has come back to life and is headed to the edge of the earth with will turner and elizabeth swann. but nothing is quite as it seems. adventure fantasy action johnnydepp orlandobloom keiraknightley goreverbinski'

# Stemming

In [244]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [245]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [246]:
ps.stem('loved')

'love'

In [247]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\PC\AppData\Local\Temp\ipykernel_6776\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [248]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict samworthington zoesaldana sigourneyweav jamescameron'

In [249]:
# ---- IMPORTANT: drop duplicates and reset index BEFORE vectorizing ----

new_df.drop_duplicates(subset='title', inplace=True)
new_df.reset_index(drop=True, inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_6776\1555759805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(subset='title', inplace=True)


# Vectorize

In [250]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words = 'english')

In [251]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [254]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [255]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

## Calculating Distance (cosine_similarity)

In [258]:
cosine_similarity(vectors).shape

(4797, 4797)

In [259]:
similarity = cosine_similarity(vectors)

In [260]:
similarity[0]

array([1.        , 0.13957263, 0.08206099, ..., 0.        , 0.        ,
       0.        ])

In [261]:
# gives you the top 5 most similar movies to the first movie (Avatar), excluding itself.

sorted(list(enumerate(similarity[0])), reverse = True, key=lambda x:x[1])[1:6]

[(2993, 0.25854384499750954),
 (4396, 0.24618298195866548),
 (61, 0.23262105259961768),
 (1438, 0.2279211529192759),
 (972, 0.22727272727272724)]

# Recommendation Function

In [264]:
# mapping title -> index (normalized to lowercase)
title_to_index = pd.Series(new_df.index, index=new_df['title'].str.lower()).to_dict()


In [265]:
# ---- robust recommend function ----
def recommend(movie, topn=5, fuzzy=False):
    movie_key = movie.strip().lower()
    if movie_key not in title_to_index:
        if fuzzy:
            # use difflib for simple fuzzy matching
            matches = get_close_matches(movie_key, title_to_index.keys(), n=3, cutoff=0.6)
            if matches:
                best = matches[0]
                print(f"Movie not found exactly. Using close match: '{best}'")
                movie_key = best
            else:
                print("Movie not found and no close match found.")
                return
        else:
            print("Movie not found! Try `recommend('Your Title', fuzzy=True)` for fuzzy matches.")
            return

    idx = title_to_index[movie_key]
    distances = similarity[idx]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1: topn+1]

    for rank, (movie_idx, score) in enumerate(movies_list, start=1):
        print(f"{rank}. {new_df.iloc[movie_idx]['title']} (score: {score:.3f})")


###  Example usage:

In [266]:
recommend('John Carter')          # exact match


1. Krrish (score: 0.334)
2. Stripes (score: 0.315)
3. Get Carter (score: 0.312)
4. The Other Side of Heaven (score: 0.309)
5. The Legend of Hercules (score: 0.304)


In [267]:
recommend('John Carte', fuzzy=True)  # fuzzy match for typos


Movie not found exactly. Using close match: 'john carter'
1. Krrish (score: 0.334)
2. Stripes (score: 0.315)
3. Get Carter (score: 0.312)
4. The Other Side of Heaven (score: 0.309)
5. The Legend of Hercules (score: 0.304)


In [268]:
recommend('Shanghai Calling')  

1. Killers (score: 0.290)
2. Should've Been Romeo (score: 0.270)
3. I Served the King of England (score: 0.265)
4. Bella (score: 0.259)
5. Highlander (score: 0.258)
